# Google Scholar
https://scholar.google.com/
Google Scholar provides a simple way to broadly search for scholarly literature. From one place, you can search across many disciplines and sources: articles, theses, books, abstracts and court opinions, from academic publishers, professional societies, online repositories, universities and other web sites. Google Scholar helps you find relevant work across the world of scholarly research

# Langchain

### Agents
https://python.langchain.com/v0.1/docs/modules/agents/
https://python.langchain.com/v0.1/docs/modules/agents/agent_types/

### Semantic Scholar
https://python.langchain.com/v0.2/docs/integrations/tools/semanticscholar/

### Google Genai
https://python.langchain.com/v0.1/docs/integrations/chat/google_generative_ai/

# Vertexai
https://cloud.google.com/vertex-ai/docs/general/custom-service-account

# GeminiAI
https://ai.google.dev/gemini-api/docs/api-key

In [1]:
%pip install --upgrade --quiet  semanticscholar langchainhub google-cloud-aiplatform langchain-google-genai langchain langchain-community langchain-experimental langchain-openai langchain-pinecone langchain-text-splitters -q --user

Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import dotenv_values

In [2]:
config = dotenv_values(".env")

In [4]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from google.oauth2 import service_account
import vertexai
import json
import os


In [5]:
openai_api_key = config["KEY_OPENAI"]
os.environ["GEMINI_API_KEY"] = config['GOOGLE_API_KEY']
# os.environ["GOOGLE_API_KEY"] = config.get("GOOGLE_API_KEY")
# os.environ["GOOGLE_CSE_ID"] = config.get("GOOGLE_CSE_ID")

In [6]:
with open("keys/complete-tube-421007-9a7c35cd44e2.json") as source:
    info = json.load(source)

vertex_credentials = service_account.Credentials.from_service_account_info(info)
vertexai.init(
    project=config["PROJECT"],
    location=config["REGION"],
    credentials=vertex_credentials,
)
google_api_key = config["GEMINI-API-KEY"]
os.environ["GEMINI_API_KEY"] = google_api_key

In [7]:
# Open AI Prompt
instructions = """You are an expert researcher."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

In [8]:
# Generic Prompt

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert researcher.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [31]:
llm = ChatOpenAI(temperature=0, api_key= openai_api_key, model="gpt-4o")

In [9]:
llm = ChatGoogleGenerativeAI(
                    model="gemini-1.5-pro-001", credentials=vertex_credentials, google_api_key=config['GOOGLE_API_KEY'], max_output_tokens=8192

                )

In [32]:
from langchain_community.tools.semanticscholar.tool import SemanticScholarQueryRun
from langchain_community.utilities.semanticscholar import SemanticScholarAPIWrapper

api_wrapper = SemanticScholarAPIWrapper(doc_content_chars_max=1000, top_k_results=5)
tools = [SemanticScholarQueryRun(api_wrapper=api_wrapper)]

### https://python.langchain.com/v0.1/docs/modules/agents/agent_types/tool_calling/

In [33]:
# Open AI
#agent = create_openai_functions_agent(llm, tools, prompt)
# generic
agent = create_tool_calling_agent(llm, tools, prompt)

In [35]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

In [36]:
response = agent_executor.invoke(
    {
        "input": """query: What is the SAPHO syndrome and what are the symptoms?
                    Instruction for output:
                    - Show me a list of papers and techniques. 
                    - Based on your findings write a description, main symptoms and how to treat them
                    - Create 3 different sections: List of papers, Description/ symptoms and treatment. 
                    - Use search tool"""
    }
)



> Entering new AgentExecutor chain...

Invoking: `semanticscholar` with `{'query': 'SAPHO syndrome symptoms and treatment'}`


Published year: 2022
Title: Exceptional response of skin symptoms to secukinumab treatment in a patient with SAPHO syndrome: Case report and literature review
Authors: Q. Ji, Qing Wang, Wenping Pan, Yanfeng Hou, Xiu-hong Wang, Lin Bian, Zhankui Wang
Astract: Rationale: SAPHO syndrome is a rare clinical entity characterized by a wide range of dermatological and musculoskeletal manifestations. Treatment strategies are not standardized. Palmoplantar pustulosis (PPP) is the most common rash in patients with SAPHO syndrome. Patient concerns: A 24-year-old Chinese woman with no relevant medical or familial history had a 1-year history of cutaneous lesions with PPP and pain in the sternoclavicular joint. Diagnosis: Based on the diagnostic criteria for SAPHO syndrome proposed by Nguyen et al in 2012, we diagnosed SAPHO syndrome with severe PPP as the predominant mani

In [37]:
response.keys()

dict_keys(['input', 'output'])

In [38]:
print(response['output'])

### List of Papers

1. **Exceptional response of skin symptoms to secukinumab treatment in a patient with SAPHO syndrome: Case report and literature review**
   - **Authors**: Q. Ji, Qing Wang, Wenping Pan, Yanfeng Hou, Xiu-hong Wang, Lin Bian, Zhankui Wang
   - **Published Year**: 2022
   - **Abstract**: SAPHO syndrome is a rare clinical entity characterized by a wide range of dermatological and musculoskeletal manifestations. Treatment strategies are not standardized. Palmoplantar pustulosis (PPP) is the most common rash in patients with SAPHO syndrome.

2. **Thrombotic manifestations in SAPHO syndrome. Review of the literature.**
   - **Authors**: T. Carranco-Medina, C. Hidalgo-Calleja, I. Calero-Paniagua, M. D. Sánchez-González, A. Quesada-Moreno, R. Usategui-Martín, L. Pérez-Garrido, S. Gómez-Castro, C. Montilla-Morales, O. Martínez-González, J. del Pino-Montes
   - **Published Year**: 2015
   - **Abstract**: None

3. **SAPHO syndrome after COVID-19 vaccination complicated with th

In [30]:
print(response['input'])

query: What is the SAPHO syndrome and what are the symptoms?
                    Instruction for output:
                    - Show me a list of papers and techniques. 
                    - Based on your findings write a description, main symptoms and how to treat them
                    - Create 3 different sections: List of papers, Description/ symptoms and treatment. 
                    - Use search tool
